In [37]:
import gzip
import pickle 
import mplhep as hep
hep.style.use("CMS")
import matplotlib
matplotlib.rc('font', size=15)
import numpy as np
import torch 
import dgl
import pandas as pd
import numpy as np
import plotly.express as px
sys.path.append("/afs/cern.ch/work/m/mgarciam/private/mlpf/")
from sklearn.cluster import DBSCAN
from torch_scatter import scatter_max, scatter_add, scatter_mean
from src.layers.inference_oc import get_clustering, obtain_intersection_values
from scipy.optimize import linear_sum_assignment
import numpy as np
import matplotlib.pyplot as plt

In [47]:
from src.utils.inference.pandas_helpers import open_mlpf_dataframe
list_all_df_hdb = []
path_hgcal = "/eos/user/m/mgarciam/EVAL_REPOS/Tracking_wcoc/models/test/showers_df_evaluation/0_0_0.pt"
sd_hgb, matched_hgb = open_mlpf_dataframe(path_hgcal, False)

In [45]:
from src.utils.inference.pandas_helpers import open_mlpf_dataframe
list_all_df_hdb = []
path_hgcal = "/eos/user/m/mgarciam/EVAL_REPOS/Tracking_wcoc/models/180324_Zcard_v_full/showers_df_evaluation/0_0_0.pt"
sd_hgb1, matched_hgb1 = open_mlpf_dataframe(path_hgcal, False)

In [49]:
sd_hgb1[0:10]

,reco_showers_E,true_showers_E,pred_showers_E,e_pred_and_truth
0,54.0,1.481437,54.0,54.0
1,48.0,1.871253,48.0,48.0
2,64.0,6.622914,64.0,64.0
3,45.0,2.206414,45.0,45.0
4,71.0,2.472970,70.0,70.0
5,38.0,1.503932,38.0,38.0
6,53.0,7.976103,53.0,53.0
7,60.0,15.701566,60.0,60.0
8,37.0,2.592102,37.0,37.0
9,42.0,2.449077,42.0,42.0


In [48]:
sd_hgb[0:10]

,reco_showers_E,true_showers_E,pred_showers_E,e_pred_and_truth
0,54.0,1.481437,54.0,54.0
1,48.0,1.871253,48.0,48.0
2,64.0,6.622914,64.0,64.0
3,45.0,2.206414,45.0,45.0
4,71.0,2.472970,70.0,70.0
5,38.0,1.503932,38.0,38.0
6,53.0,7.976103,53.0,53.0
7,60.0,15.701566,60.0,60.0
8,37.0,2.592102,37.0,37.0
9,42.0,2.449077,42.0,42.0


In [23]:
from src.logger.plotting_tools import shuffle_truth_colors

In [93]:
from src.logger.plotting_tools import shuffle_truth_colors
label_print = dic["graph"].ndata['particle_number'].view(-1).clone()
mask = label_print==4
tidx = label_print.view(-1)
features =  dic["graph"].ndata["e_hits"]
pos_hits =dic["graph"].ndata["pos_hits_xyz"]
data = {
            "X": pos_hits[:, 0].view(-1, 1).detach().cpu().numpy(),
            "Y": pos_hits[:, 1].view(-1, 1).detach().cpu().numpy(),
            "Z": pos_hits[:, 2].view(-1, 1).detach().cpu().numpy(),
            "tIdx": tidx.view(-1, 1).detach().cpu().numpy(),
            "features": features.view(-1, 1).detach().cpu().numpy(),
        }
hoverdict = {}
# if hoverfeat is not None:
#     for j in range(hoverfeat.shape[1]):
#         hoverdict["f_" + str(j)] = hoverfeat[:, j : j + 1]
#     data.update(hoverdict)

# if nidx is not None:
#     data.update({"av_same": av_same})

df = pd.DataFrame(
np.concatenate([data[k] for k in data], axis=1),
columns=[k for k in data],
)
df["orig_tIdx"] = df["tIdx"]
rdst = np.random.RandomState(1234567890)  # all the same
#shuffle_truth_colors(df, "tIdx", rdst)

# hover_data = ["orig_tIdx", "idx"] + [k for k in hoverdict.keys()]
# if nidx is not None:
#     hover_data.append("av_same")
fig = px.scatter_3d(
df,
x="X",
y="Y",
z="Z",
color="tIdx",
size="features",
# hover_data=hover_data,
template="plotly_dark",
color_continuous_scale=px.colors.sequential.Rainbow,
)
fig.update_traces(marker=dict(line=dict(width=0)))

In [145]:

store_image(label=False, cluster=False, pandora=True)

In [ ]:
store_image(label=True, cluster=False)

In [49]:
store_image(label=True, cluster=False)
store_image(label = False, cluster=False)
store_image(label=False, cluster=True)
store_image(label = True, cluster=True)
store_image(label = True, true_false=True)

In [10]:
def plot_iou_matrix(iou_matrix, image_path):
    iou_matrix = torch.transpose(iou_matrix[1:, :], 1, 0)
    fig, ax = plt.subplots()
    iou_matrix = iou_matrix.detach().cpu().numpy()
    ax.matshow(iou_matrix, cmap=plt.cm.Blues)
    for i in range(0, iou_matrix.shape[1]):
        for j in range(0, iou_matrix.shape[0]):
            c = np.round(iou_matrix[j, i], 1)
            ax.text(i, j, str(c), va="center", ha="center")

In [11]:
def store_image(label=False, cluster = False, pandora = False, true_false=False):
    if label:
        tidx = np.reshape(labels, (-1,1)) 
    elif pandora:
        if true_false:
            tidx = np.reshape(label_matched_pandora, (-1,1)) 
        else:
            tidx = np.reshape(labels_pandora, (-1,1))
    elif true_false:
        tidx = np.reshape(label_matched, (-1,1)) 
    else:
        tidx = dic["graph"].ndata['particle_number'].view(-1,1)
    features =  torch.log(dic["graph"].ndata["e_hits"])+10
    pos_hits =dic["graph"].ndata["pos_hits_xyz"]
    if cluster:
        pos_hits = X
    data = {
                "X": pos_hits[:, 0].view(-1, 1).detach().cpu().numpy(),
                "Y": pos_hits[:, 1].view(-1, 1).detach().cpu().numpy(),
                "Z": pos_hits[:, 2].view(-1, 1).detach().cpu().numpy(),
                "tIdx": tidx,
                "features": features.view(-1, 1).detach().cpu().numpy(),
            }
    hoverdict = {}
    # if hoverfeat is not None:
    #     for j in range(hoverfeat.shape[1]):
    #         hoverdict["f_" + str(j)] = hoverfeat[:, j : j + 1]
    #     data.update(hoverdict)

    # if nidx is not None:
    #     data.update({"av_same": av_same})

    df = pd.DataFrame(
    np.concatenate([data[k] for k in data], axis=1),
    columns=[k for k in data],
    )
    df["orig_tIdx"] = df["tIdx"]
    rdst = np.random.RandomState(1234567890)  # all the same
    #shuffle_truth_colors(df, "tIdx", rdst)

    # hover_data = ["orig_tIdx", "idx"] + [k for k in hoverdict.keys()]
    # if nidx is not None:
    #     hover_data.append("av_same")
    fig = px.scatter_3d(
    df,
    x="X",
    y="Y",
    z="Z",
    color="tIdx",
    size="features",
    # hover_data=hover_data,
    template="plotly_dark",
    color_continuous_scale=px.colors.sequential.Rainbow,
    )
    fig.update_traces(marker=dict(line=dict(width=0)))
    if pandora and true_false:
        path= "/eos/user/m/mgarciam/EVAL_REPOS/mlpf/pandora_label_true_false.html"
    elif true_false:
        path= "/eos/user/m/mgarciam/EVAL_REPOS/mlpf/model_label__true_false.html"
    elif label and not cluster:
        path= "/eos/user/m/mgarciam/EVAL_REPOS/mlpf/model_label.html"
    elif pandora and not cluster:
        path= "/eos/user/m/mgarciam/EVAL_REPOS/mlpf/pandora_label.html"
    elif label and cluster:
        path= "/eos/user/m/mgarciam/EVAL_REPOS/mlpf/model_label_cluster.html"
    elif not label and  cluster:
        path= "/eos/user/m/mgarciam/EVAL_REPOS/mlpf/true_label_cluster.html"
    else:
        path= "/eos/user/m/mgarciam/EVAL_REPOS/mlpf/true_label.html"
    fig.write_html(path)